In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# Cargamos el dataframe en df
df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [3]:
# Exploramos el df buscando columnas con muchos NA
df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [4]:
# Eliminamos columnas con demasiados NA (más del 40%)
umbral = 0.4
to_drop2 = set()
for col in df:
    if df[col].isna().mean() > umbral:
        to_drop2.add(col)
print("Variables a eliminar por alta cantidad de NA:")
print(to_drop2)
df = df.drop(columns=list(to_drop2))

Variables a eliminar por alta cantidad de NA:
{'MiscFeature', 'PoolQC', 'MasVnrType', 'Fence', 'Alley', 'FireplaceQu'}


In [5]:
# Rellenamos NA con la media en vbles numericas y con la moda en categoricas
for col in df:
    if df[col].dtypes == "int64":
        df[col] = df[col].fillna((df[col].mean()))
    elif df[col].dtypes == "float64":
        df[col] = df[col].fillna(df[col].mean())
    elif df[col].dtypes == "object":
        df[col] = df[col].fillna(df[col].mode().iloc[0])

# Normalizamos las variables numericas excepto la objetivo
df_num = df.drop(["SalePrice", "Id"], axis = 1)
df_num = df_num.select_dtypes(include=['int64', 'float64'])
df_num_norm = (df_num - df_num.min()) / (df_num.max() - df_num.min())
df_num_norm = pd.concat([df_num_norm,df[["SalePrice","Id"]]], axis = 1)

In [6]:
# Se eliminan variables muy correlacionadas entre sí.

import pandas as pd
import numpy as np

# 1. Calcular matriz de correlación solo con variables numéricas
corr = df_num_norm.corr()

# 2. Correlación absoluta con la variable objetivo 'SalePrice'
corr_target = corr['SalePrice'].abs()

# 3. Crear conjunto para guardar variables a eliminar
to_drop = set()

# 4. Iterar por pares de variables (solo la mitad superior de la matriz)
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        var1 = corr.columns[i]
        var2 = corr.columns[j]
        val = corr.iloc[i, j]

        # 5. Si están altamente correlacionadas (>|0.8|)
        if abs(val) > 0.8:
            # 6. Comparar su correlación con 'SalePrice' y eliminar la más débil
            if corr_target[var1] < corr_target[var2]:
                to_drop.add(var1)
            else:
                to_drop.add(var2)

# 7. Mostrar las variables a eliminar
print("Variables a eliminar por alta correlación:")
print(to_drop)

# 8. Eliminar esas variables del DataFrame numerico normalizado
df_num_norm = df_num_norm.drop(columns=list(to_drop))

Variables a eliminar por alta correlación:
{'1stFlrSF', 'TotRmsAbvGrd', 'GarageArea'}


In [7]:
# El conjunto de train solo tiene 1460 entradas y 37 variables object
# Las 37 object se multiplicaran tras el one hot 

# Para tener suficientes grados de libertad habrá que hacer selección
df_cat = df.select_dtypes(include=['object'])
df_des = df_cat.describe(include= "object")

# Se eliminan las categoricas con valor con frequencia superior a umbral (0.8)
to_drop3 = set()
for col in df_des:
    if df_des[col].iloc[3]/len(df) > umbral:
        to_drop3.add(col)
print("Variables a eliminar por alta frecuencia:")
print(to_drop3)
df_cat = df_cat.drop(columns=list(to_drop3))

Variables a eliminar por alta frecuencia:
{'BldgType', 'HeatingQC', 'Utilities', 'LotShape', 'RoofMatl', 'LotConfig', 'LandSlope', 'CentralAir', 'LandContour', 'Street', 'BsmtQual', 'Functional', 'Heating', 'Condition1', 'ExterCond', 'BsmtFinType2', 'Electrical', 'GarageQual', 'BsmtCond', 'RoofStyle', 'GarageCond', 'GarageFinish', 'Foundation', 'GarageType', 'SaleCondition', 'MSZoning', 'PavedDrive', 'HouseStyle', 'Condition2', 'ExterQual', 'SaleType', 'BsmtExposure', 'KitchenQual'}


In [8]:
df_concat = pd.concat([df_num_norm, df_cat], axis = 1)

In [9]:
# Creamos las dummies de las categoricas
dummies = pd.get_dummies(df_concat)
dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 97 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MSSubClass            1460 non-null   float64
 1   LotFrontage           1460 non-null   float64
 2   LotArea               1460 non-null   float64
 3   OverallQual           1460 non-null   float64
 4   OverallCond           1460 non-null   float64
 5   YearBuilt             1460 non-null   float64
 6   YearRemodAdd          1460 non-null   float64
 7   MasVnrArea            1460 non-null   float64
 8   BsmtFinSF1            1460 non-null   float64
 9   BsmtFinSF2            1460 non-null   float64
 10  BsmtUnfSF             1460 non-null   float64
 11  TotalBsmtSF           1460 non-null   float64
 12  2ndFlrSF              1460 non-null   float64
 13  LowQualFinSF          1460 non-null   float64
 14  GrLivArea             1460 non-null   float64
 15  BsmtFullBath         

In [10]:
# Creamos el modelo para entrenar
# Quitamos Id del conjunto de datos
x_train = dummies.drop(columns=['Id', "SalePrice"])
y_train = dummies["SalePrice"]

# Creamos el modelo SVM de regresion
from sklearn.svm import SVR
svr = SVR()
# Creamos el param_grid para un SVM de regresion
param_grid = {
    'C': [1000, 10000, 100000],
    'epsilon': [0.01, 0.1, 0.5, 1],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']  # solo se aplica si kernel='rbf'
}

# Creamos el gridsearch
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(
    estimator=svr,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1
)

grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_

print("The best model is: ",best_model)
print("With R2 value of: ",grid_search.best_score_)

The best model is:  SVR(C=100000, epsilon=1)
With R2 value of:  0.844399301315549


In [11]:
# Hacemos prediccion
# Cargamos los datos de test
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
# Son 50% de datos de train y 50% de test
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [12]:
# Realizamos las mismas transformaciones que sobre el conjunto de train
ids = test['Id'].copy()
for col in test:
    if test[col].dtypes == "int64":
        test[col] = test[col].fillna(test[col].mean())
    elif test[col].dtypes == "float64":
        test[col] = test[col].fillna(test[col].mean())
    elif test[col].dtypes == "object":
        test[col] = test[col].fillna(test[col].mode().iloc[0])

# Se eliminan las variables que se eliminaron en train
test = test.drop(columns=list(to_drop2))

# Normalizamos las variables numericas
test_num = test.select_dtypes(include=['int64', 'float64'])
test_num_norm = (test_num - test_num.min()) / (test_num.max() - test_num.min())
test_num_norm = test_num_norm.drop(["Id"], axis = 1)
test_num_norm = test_num_norm.drop(columns=list(to_drop))
test_num_norm = pd.concat([ids, test_num_norm], axis=1)

# Filtramos las variables categoricas eliminadas en train
test_cat = test.select_dtypes(include=['object'])
test_cat = test_cat.drop(columns=list(to_drop3))

In [13]:
# Se concatenan ambos test,
test_concat = pd.concat([test_num_norm, test_cat], axis = 1)

In [14]:
dummies_test = pd.get_dummies(test_concat)

In [15]:
# Alineamos train con test para que tengan la misma dimension
dummies, dummies_test = dummies.align(dummies_test, join='left', axis=1, fill_value=0)

In [16]:
x_test = dummies_test.drop(["SalePrice","Id"], axis = 1)

In [17]:
# Realizamos prediccion
y_pred = best_model.predict(x_test)
submit = pd.DataFrame()
submit["Id"] = test["Id"]
submit["SalePrice"] = y_pred
submit.to_csv("submission.csv", index = False)